<a href="https://colab.research.google.com/github/YuxingW/machine_learning/blob/main/hw7/Scrapping_from_Politifact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4 

In [2]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup  

url = "https://www.politifact.com/factchecks/list/?page={pgno}&speaker={speaker}"

records = []  

for i in range(1,5):
  fUrl = url.format(pgno=str(i), speaker="donald-trump")
  r = requests.get(fUrl)
  soup = BeautifulSoup(r.text, 'html.parser')  
  results = soup.find_all('div', attrs={'class':'m-statement__body'})
  for result in results:
    date = result.find('footer',attrs={'class':'m-statement__footer'}).text.split("•")[1].rstrip("\n")  
    reporter = result.find('footer',attrs={'class':'m-statement__footer'}).text.split("•")[0].replace("\nBy ","")
    statement =  result.find('div', attrs = {'class':'m-statement__quote'}).find('a').text.replace("\n","")
    article_url =  result.find('a')['href']
    verdict = result.find('img', attrs = {'class':'c-image__thumb'}, alt=True).attrs['alt']
    records.append(( date,reporter, statement, verdict, article_url))
  
df = pd.DataFrame(records, columns=['Date','Reporter', 'Statement', 'Verdict', 'Url'])  
df.head()

,Date,Reporter,Statement,Verdict,Url
0,"March 30, 2021",Jon Greenberg,Says Deborah Birx “traveled a great distance t...,half-true,/factchecks/2021/mar/30/donald-trump/donald-tr...
1,"March 24, 2021",Amy Sherman,"""We proudly handed the Biden administration th...",barely-true,/factchecks/2021/mar/24/donald-trump/what-trum...
2,"March 2, 2021",Tom Kertscher,"Says he requested “10,000 National Guardsmen” ...",false,/factchecks/2021/mar/02/donald-trump/no-proof-...
3,"March 2, 2021",Amy Sherman,HR 1 “automatically registers every welfare re...,barely-true,/factchecks/2021/mar/02/donald-trump/trump-mis...
4,"March 1, 2021",Miriam Valverde,Says Joe Biden “has effectively ordered a shut...,false,/factchecks/2021/mar/01/donald-trump/cpac-retu...


In [3]:
#Get all issues

url = "https://www.politifact.com/issues/"
issues = []
r = requests.get(url)
soup = BeautifulSoup(r.text,'html.parser')
results = soup.find_all('div', attrs={'class':'c-chyron__value'})
for result in results:
  name = result.find('a').text
  issue = result.find('a')['href'].replace("/","")
  issues.append((name, issue))

In [4]:
url = "https://www.politifact.com/factchecks/list/?page={pgno}&category={category}"

records = []  

for i in range(1,2):
  for issue, issue_url in issues[:5]:
    fUrl = url.format(pgno=str(i), category=issue_url)
    r = requests.get(fUrl)
    soup = BeautifulSoup(r.text, 'html.parser')  
    results = soup.find_all('article', attrs={'class':'m-statement'})
    for result in results:
      date = result.find('footer',attrs={'class':'m-statement__footer'}).text.split("•")[1].rstrip("\n")
      reporter = result.find('footer',attrs={'class':'m-statement__footer'}).text.split("•")[0].replace("\nBy","")   
      author = result.find('a',attrs={'class':'m-statement__name'}).text.replace("\n","")
      statement =  result.find('div', attrs = {'class':'m-statement__quote'}).find('a').text.replace("\n","")
      statement_descr = result.find('div', {'class':'m-statement__desc'}).text.replace("\n","")
      article_url =  result.find('a')['href']
      verdict = result.find('img', attrs = {'class':'c-image__thumb'}, alt=True).attrs['alt']
      records.append(( date, issue, reporter, author, statement, statement_descr, verdict, article_url))
  
df_sample = pd.DataFrame(records, columns=['Date', 'Issue','Reporter','Author', 'Statement', 'Description', 'Verdict', 'Url'])  
df_sample.head()

,Date,Issue,Reporter,Author,Statement,Description,Verdict,Url
0,"March 31, 2021",Abortion,Tom Kertscher,Facebook posts,“Joe Biden puts pro-life groups on domestic ex...,"stated on March 29, 2021 in a Facebook post:",barely-true,/personalities/facebook-posts/
1,"February 12, 2021",Abortion,Brandon Mulder,Greg Abbott,“Innocent lives will be saved” by ending taxpa...,"stated on January 24, 2021 in a tweet:",false,/personalities/greg-abbott/
2,"November 18, 2020",Abortion,Noah Y. Kim,Facebook posts,There is “aborted male fetus” in the Oxford-As...,"stated on November 15, 2020 in a Facebook post:",false,/personalities/facebook-posts/
3,"October 14, 2020",Abortion,Tom Kertscher,Tommy Tuberville,"Says Doug Jones ""has voted to spend our tax do...","stated on October 8, 2020 in an ad:",false,/personalities/tommy-tuberville/
4,"October 13, 2020",Abortion,Chris Nichols,Facebook posts,“Kamala Harris Tried to Put Pro-Lifers in Jail...,"stated on August 14, 2020 in a headline:",false,/personalities/facebook-posts/


In [5]:
df_sample.sample(20)

,Date,Issue,Reporter,Author,Statement,Description,Verdict,Url
89,"April 3, 2019",Agriculture,Chris Nichols,Bloggers,"Americans ""would run out of avocados in three ...","stated on April 1, 2019 in news articles and b...",mostly-true,/personalities/blog-posting/
34,"June 30, 2020",Ad Watch,Louis Jacobson,Joe Biden,“Trump lost a trade war that he started.”,"stated on June 24, 2020 in an ad:",mostly-true,/personalities/joe-biden/
70,"July 1, 2016",After the Fact,Will Doran,ACLU of North Carolina,Proposed NC law for body camera footage gives ...,"stated on June 29, 2016 in a press release:",true,/personalities/aclu-north-carolina/
28,"October 22, 2019",Abortion,Ciara O'Rourke,Association of Mature American Citizens,"AARP backed ""federal funding for Planned Paren...","stated on August 29, 2019 in a Facebook ad:",false,/personalities/association-mature-american-cit...
5,"October 9, 2020",Abortion,Miriam Valverde,Mike Pence,Says Joe Biden and Kamala Harris support abort...,"stated on October 7, 2020 in a debate:",false,/personalities/mike-pence/
29,"October 9, 2019",Abortion,Warren Fiske,Kelly McGinn,"Del. Danica Roem ""supported a controversial bi...","stated on September 20, 2019:",false,/personalities/kelly-mcginn/
1,"February 12, 2021",Abortion,Brandon Mulder,Greg Abbott,“Innocent lives will be saved” by ending taxpa...,"stated on January 24, 2021 in a tweet:",false,/personalities/greg-abbott/
52,"December 14, 2014",Afghanistan,Jon Greenberg,Dick Cheney,"Says Saddam Hussein ""had a 10-year relationshi...","stated on December 14, 2014 in comments on NBC...",false,/personalities/dick-cheney/
95,"February 7, 2019",Agriculture,Ciara O'Rourke,Viral image,"Says Stacey Abrams said ""Who needs agriculture...","stated on February 5, 2019 in a Facebook post:",pants-fire,/personalities/viral-image/
15,"March 16, 2020",Abortion,Samantha Putterman,Bloggers,Says Nancy Pelosi was “caught trying to includ...,"stated on March 12, 2020 in a story:",false,/personalities/blog-posting/
